In [1]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [2]:
import transform_db
from transform_db import load_clean
from transform_db import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3


pd.set_option("display.max_columns",None)

In [3]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
load = load_clean(path)
agg_boxscores = load.agg_boxscores_raw()

In [4]:
conn = sqlite3.connect(path)

clean = transform(conn,2013,2023)
data = clean.load_team_data()
data = clean.clean_team_data(data)
data = data.dropna(subset='PCT_PTS_2PT')
data = clean.convert_pcts(data)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_df = data.loc[:,data.columns[9:]]
scaled_array = scaler.fit_transform(scaled_df)
scaled = pd.DataFrame(scaled_array,columns=scaled_df.columns)

In [6]:
add_columns = data.loc[:,data.columns[:9]]
data[scaled.columns] = scaled.values

# Create Input Array

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(data['TEAM_ABBREVIATION'])

data['TEAM_ABBREVIATION'] = encoder.transform(data['TEAM_ABBREVIATION'])

In [8]:
filter = ['SEASON',"TEAM_ID",'TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP']
data_clean = data[data.columns.difference(filter)]

In [9]:
X_list = []
y_list = []
window_size = 10

for i in range(len(data_clean['TEAM_ABBREVIATION'].unique())):
    team = data_clean.loc[data_clean['TEAM_ABBREVIATION'] == i]
    team_label = team['WL'].copy()
    team = team.drop('WL',axis=1)

    team_np = team.to_numpy()
    team_label = team_label.to_numpy()

    #temp_X = []
    #temp_y = []
    
    for i in range(len(team_np)-window_size):        
      row = [a for a in team_np[i:i+window_size]]
      X_list.append(row)

      label = team_label[i+window_size]
      y_list.append(label)

X_list = np.array(X_list)
y_list = np.array(y_list)

X_list.shape, y_list.shape

((23422, 10, 34), (23422,))

In [10]:
from model import ModelConfig

2024-02-01 23:22:37.308292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 23:22:37.308326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 23:22:37.308798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 23:22:37.311852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 23:22:37.724770: W tensorflow/compiler/tf2

In [11]:
model_config = ModelConfig(X_list,y_list,[.75,.10,.15])

In [12]:
X_train,y_train,X_test,y_test,X_val,y_val = model_config.train_test_val()

In [13]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

model = model_config.create_sequential()
model.add(LSTM(64))
model.add(Dense(8,'relu'))
model.add(Dense(1,'sigmoid'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.00001), metrics=[RootMeanSquaredError()])

2024-02-01 23:22:38.138124: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 23:22:38.155898: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 23:22:38.155928: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 23:22:38.159314: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 23:22:38.159343: I external/local_xla/xla/stream_executor

In [14]:
import tensorflow as tf
tf.sysconfig.get_build_info()

OrderedDict([('cpu_compiler', '/usr/lib/llvm-17/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '12.2'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [15]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
model.optimizer